__AASB_Motifs__

Motifs probabilísticos baseados num alinhamento múltiplo

In [3]:
import re
from math import log
from threading import local
from typing import List, Type
from unittest import result

In [9]:
lista_seqs = ['ATTG','ATCG','ATTC','ACTC']

seqs2 = """HEM13 CCCATTGTTCTC
HEM13 TTTCTGGTTCTC
HEM13 TCAATTGTTTAG
ANB1 CTCATTGTTGTC
ANB1 TCCATTGTTCTC
ANB1 CCTATTGTTCTC
ANB1 TCCATTGTTCGT
ROX1 CCAATTGTTTTG"""

seqs2 = [x.split()[1] for x in seqs2.splitlines()]

In [10]:
def pwm(lista_seqs: List[str], pseudo: int = 0, printing: bool = True):
    ''' 
    Funcao que recebe uma lista de sequencias de tamanho igual o valor da pseucocontagem e o tipo de printing desejado. Devolve o perfil PWM das mesmas. \n
        
    Inputs:
        :lista_seqs: Lista de sequências
        :type lista_seqs: List[str]
        :pseudo: Valor da pseudocontagem do PWM (por omissão 0)
        :type pseudo: int
        :printing: Modo de retorno da matriz
        :type printing: bool
        
    Returns:
        :return PWM: PWM das sequências
        :rtype PWM: array
    '''
    if type(lista_seqs) not in [list]:
        raise TypeError ('A sua lista de sequências não é uma lista')

    for i in lista_seqs:
        if type(i) not in [str]:
            raise TypeError ('A sua lista de sequências possui valores que não são strings.')
    
    if len(lista_seqs) == 1:
        raise TypeError ('A sua lista de sequências deve ter mais do que uma sequência.')

    
    num_seqs = len(lista_seqs)
    result = []
    for z in zip(*lista_seqs):
        result.append({k: (z.count(k) + pseudo) / (num_seqs + (pseudo * 4)) for k in 'ACGT'})
    # o valor de pseudo é multiplicado pelo valor das bases, no caso de ser proteina pelos aminoacidos
    
    #print
    if printing == True:
        bases = sorted(result[0].keys())    # Do primeiro dicionario da lista de listas, tirar as keys (a,c,g,t)
        tab = [[f"{p[b]:-5.2f}" for b in bases] for p in result]    # para cada linha em perfil adicionar um (a,c,g,t)
        for p in zip(*([bases] + tab)):
            print (*p)
    else:
        return result

In [27]:
p = pwm(lista_seqs, pseudo = 0, printing = True)

A  1.00  0.00  0.00  0.00
C  0.00  0.25  0.25  0.50
G  0.00  0.00  0.00  0.50
T  0.00  0.75  0.75  0.00


In [13]:
def pssm(lista_seqs: List[str], pseudo: int = 1, printing: bool = False):
    ''' 
    Funcao que recebe uma lista de sequencias de tamanho igual e a pseudocontagem (pseudo). Devolve o perfil PSSM das mesmas. \n
    
    Inputs:
        :lista_seqs: Lista de sequências
        :type lista_seqs: List[str]
        :pseudo: Valor da pseudocontagem do PSSM (por omissão 0)
        :type pseudo: int
        
    Returns:
        :return PSSM: PSSM das sequências
        :rtype PSSM: array
    '''

    if type(lista_seqs) not in [list]:
        raise TypeError ('A sua lista de sequências não é uma lista')

    for i in lista_seqs:
        if type(i) not in [str]:
            raise TypeError ('A sua lista de sequências possui valores que não são strings.')
    
    if len(lista_seqs) == 1:
        raise TypeError ('A sua lista de sequências deve ter mais do que uma sequência.')

    if pseudo == 0:
        raise TypeError ('A pseudocontagem do algoritmo PSSM não pode ser 0')

    num_seqs = len(lista_seqs)
    result = []
    for z in zip(*lista_seqs):
        result.append({k: log((z.count(k) + pseudo) * 4 / (num_seqs + (pseudo * 4)), 2) for k in "ACGT"})
    
    # print
    if printing == True:
        bases = sorted(result[0].keys())    # Do primeiro dicionario da lista de listas, tirar as keys (a,c,g,t)
        tab = [[f"{p[b]:-5.2f}" for b in bases] for p in result]    # para cada linha em perfil adicionar um (a,c,g,t)
        for p in zip(*([bases] + tab)):
            print(*p)
    else:
        return result

def seq_mais_prob(lista_seqs: List[str], seq: str, pseudo: int = 0):

    '''
    Funcao que recebe uma lista de sequencias com o mesmo tamanho, a pseudocontagem (pseudo) e uma sequencia maior. Esta funcao vai realizar
    um perfil pwm e descobrir de entre as sequências na lista de sequencias a mais provavel de ocorrer. \n
    Inputs:
        :lista_seqs: Lista de sequências
        :type lista_seqs: List[str]
        :seq: Sequência DNA
        :type seq: string
        :pseudo: Valor da pseudocontagem para o PWM (por omissão 0)
        :type pseudo: int
        
    Returns:
        :return seq_mais_provavel: Sequência mais provavel com base nos inputs
        :rtype PWM: str 
    '''

    if type(lista_seqs) not in [list]:
        raise TypeError ('Deve dar input de uma lista de strings')

    def pwm(lista_seqs,pseudo=0):
        num_seqs = len(lista_seqs)
        result = []
        for z in zip(*lista_seqs):
            result.append({k: (z.count(k) + pseudo) / (num_seqs + (pseudo * 4)) for k in "ACGT"})
        return result
    
    def prob_seq(seq,pwm):
        assert len(seq)==len(pwm), "Tamanhos diferentes entre 'seq' e 'pwm'!"
        result = 1
        for s in range(len(seq)):
            result *= pwm[s][seq[s]]
        return result
    
    def mais_prob(seq,P):
        chave=[]
        valor=[]
        for s in re.findall('(?=(....))', seq):
            chave.append(s)
            valor.append(prob_seq(s, pwm = P))
        dic= dict(zip(chave, valor))
        return f'{max(dic, key=dic.get)}', f'{dic[max(dic, key=dic.get)]}'
        # print(f'Sequencia mais provavel:{max(dic, key=dic.get)}')
        # print(f'Probabilidade da sequencia mais provavel: {dic[max(dic, key=dic.get)]}')
        
    
    P=pwm(lista_seqs,pseudo)
    return mais_prob(seq,P)

In [14]:
pssm(lista_seqs, pseudo = 1, printing = False)

[{'A': 1.3219280948873624, 'C': -1.0, 'G': -1.0, 'T': -1.0},
 {'A': -1.0, 'C': 0.0, 'G': -1.0, 'T': 1.0},
 {'A': -1.0, 'C': 0.0, 'G': -1.0, 'T': 1.0},
 {'A': -1.0, 'C': 0.5849625007211562, 'G': 0.5849625007211562, 'T': -1.0}]

In [60]:
def seq_mais_prob(lista_seqs: List[str], seq: str, pseudo: int = 0):

    '''
    Funcao que recebe uma lista de sequencias com o mesmo tamanho, a pseudocontagem (pseudo) e uma sequencia maior. Esta funcao vai realizar
    um perfil pwm e descobrir de entre as sequências na lista de sequencias a mais provavel de ocorrer. \n
    Inputs:
        :lista_seqs: Lista de sequências
        :type lista_seqs: List[str]
        :seq: Sequência DNA
        :type seq: string
        :pseudo: Valor da pseudocontagem para o PWM (por omissão 0)
        :type pseudo: int
        
    Returns:
        :return seq_mais_provavel: Sequência mais provavel com base nos inputs
        :rtype PWM: str 
    '''

    if type(lista_seqs) not in [list]:
        raise TypeError ('Deve dar input de uma lista de strings')

    def pwm(lista_seqs,pseudo=0):
        num_seqs = len(lista_seqs)
        result = []
        for z in zip(*lista_seqs):
            result.append({k: (z.count(k) + pseudo) / (num_seqs + (pseudo * 4)) for k in "ACGT"})
        return result
    
    def prob_seq(seq,pwm):
        assert len(seq)==len(pwm), "Tamanhos diferentes entre 'seq' e 'pwm'!"
        result = 1
        for s in range(len(seq)):
            result *= pwm[s][seq[s]]
        return result
    
    def mais_prob(seq,P):
        chave=[]
        valor=[]
        for s in re.findall('(?=(....))', seq):
            chave.append(s)
            valor.append(prob_seq(s, pwm = P))
        dic= dict(zip(chave, valor))
        return f'{max(dic, key=dic.get)}', f'{dic[max(dic, key=dic.get)]}'
        
    P=pwm(lista_seqs,pseudo)
    return mais_prob(seq,P)

In [61]:
seq_mais_prob(lista_seqs,seq = "TATTTACTCTTTTAGATGTTCTGGACTCAATTGCTCACTCTTCTCTACTCGTTTAGATGATAGTAGCGCTATTGACTCCGATCTTACTCTTTATGATCGATCGATGCACTCATCGTACGTAGATGCTTTT", pseudo = 0)

('ATTG', '0.28125')